RAG chatbot 
Version 1.0 

Import Libraries

In [60]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.retrievers import MultiQueryRetriever
import chromadb

# For formatting output
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

LOAD PDF

In [61]:
# Load PDF
doc_path = "TaskFlow.pdf"
if doc_path:
    loader = UnstructuredPDFLoader(file_path=doc_path, languages=["deu"])
    data = loader.load()
    print(f"PDF loaded successfully: {doc_path}")
else:
    print("Upload documents")

#TODO: language detection!!!!


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


PDF loaded successfully: TaskFlow.pdf


SPLIT TEXT INTO CHUNKS

In [62]:
#Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Number of text chunks: {len(chunks)}")

Number of text chunks: 6


CREATE VECTOR DATABASE

In [63]:
# Create vector database
client = chromadb.PersistentClient(path="./chroma_data")
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="rag_chatbot_collection",
    client=client
)
print("Vector database created successfully.")


Vector database created successfully.


Set up LLM and Retrieval

In [64]:
#Set up LLM and retrieval
local_model = "llama3.2"
llm = ChatOllama(model=local_model, temperature=0.1)

# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a query rephrasing assistant for a vector database search.
    Your task is to generate **3** alternative, semantically diverse versions of the user's question. These variations should explore different terminology, structures, or angles of the original query to maximize relevant document retrieval.
    Provide only the rephrased questions, with each one separated by a newline. Do not include any introductory phrases, explanations, or the original question itself.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

Create chain

In [65]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


# Create chain (pipeline)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Chat with PDF

In [66]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))


In [67]:
# Example 1
chat_with_pdf("What is the main idea of this document?")

The main idea of this document appears to be a task or project description for a software development project, likely related to a task management system. The document outlines various requirements and specifications for the project, including functional and non-functional requirements, technical frameworks, and acceptance criteria.

In [68]:
# Example 2
chat_with_pdf("What are the two options for technical implementation of this system?")

The two options for technical implementation of this system are:

1. Implementierung als Webanwendung (Spring o.ä.)
2. Implementierung als Desktop Anwendung (Swing)

In [69]:
# Example 4
chat_with_pdf("What should I do? Can you give me some steps?")

Based on the provided context, it seems that you are working with a task management system called TaskFlow, and you need to implement or improve its functionality.

Here are some steps you can follow:

1. **Read and understand the requirements**: Review the document's page content, especially sections 2 (Funktionale Anforderungen) and 3 (Performance, Zuverlässigkeit, Wartbarkeit). Make sure you understand what is expected from the system.
2. **Identify areas for improvement**: Based on your understanding of the requirements, identify areas where you can improve or implement new features. This might include enhancing the user interface, improving performance, or adding new functionality.
3. **Choose an implementation approach**: You have two variants to choose from: implementing TaskFlow as a web application using Spring (or another framework). Consider which approach is more suitable for your needs and expertise.
4. **Design and implement the system**: Start designing and implementing the system according to your chosen approach. Make sure to follow the modular architecture and clear separation of concerns.
5. **Test and validate**: Implement testing and validation strategies to ensure that the system meets the requirements and works as expected. Aim for a test coverage of at least 80%.
6. **Deploy and maintain**: Once the system is implemented, deploy it and ensure its high availability (>99%). Monitor performance and fix any issues promptly.
7. **Refine and iterate**: Continuously refine and improve the system based on user feedback and new requirements.

Remember to follow best practices for software development, such as using version control, writing clean code, and documenting your implementation.